<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this notebook, I will explore, segment, and cluster the neighborhoods in the city of Toronto.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information I need to explore and cluster the neighborhoods in Toronto. I will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format. Once the data is in a structured format, I will analyze the dataset to explore and cluster the neighborhoods in the city of Toronto.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Scrape Wikipedia to obtain the Data</a>
    
2. <a href="#item2">Data Wrangling / Data Cleaning</a>
    
3. <a href="#item3">Add Lat/Lon</a>
    
4. <a href="#item4">Explore Neighborhoods in Toronto City</a>  
    
5. <a href="#item5">Analyze Each Neighborhood</a>  
    
6. <a href="#item6">Cluster Neighborhoods</a>  
    
7. <a href="#item7">Examine Clusters</a> 
    
8. <a href="#item8">Conclusion</a>  
  
</font>
</div>

Before getting the data and start exploring it, let's download all the dependencies that we will need.

In [88]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import geocoder # import geocoder
import folium # map rendering library
import requests # library to handle requests
from sklearn.cluster import KMeans # import k-means for clustering
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Scrape Wikipedia to obtain the Data

In this section, I will scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [89]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# For scraping the table, I use pandas to read the table into a pandas dataframe.
tables = pd.read_html(link)

# There are 3 tables in the Wikipedia webpage, but I am interested in the first one
postal_code_toronto = tables[0]
postal_code_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<a id='item1'></a>

## 2.  Data Wrangling / Data Cleaning

In [90]:
# Renaming first column
postal_code_toronto.rename(columns = {'Postcode':'PostalCode'}, inplace = True)

In [91]:
# Dropping rows with "Not assigned" Borough
toronto_data = postal_code_toronto[postal_code_toronto['Borough'] != 'Not assigned'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [92]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
toronto_data_clean = toronto_data.copy()
toronto_data_clean.loc[toronto_data['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = toronto_data['Borough']
toronto_data_clean.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [93]:
# More than one neighborhood can exist in one postal code area. 
# In this case, rows will be combined into one row with the neighborhoods separated with a comma
toronto_data_clean_grouped = toronto_data_clean.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).reset_index()
toronto_data_clean_grouped.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [94]:
print("Shape {}".format(toronto_data_clean_grouped.shape))

Shape (103, 3)


<a id='item3'></a>

## 3.  Add Lat/Lon

In [95]:
# initialize your variables
lats = []
lons = []

# loop through all postal codes
for postal_code in toronto_data_clean_grouped['PostalCode']:
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
    
    # append the new coordinates 
    lats.append(lat_lng_coords[0])
    lons.append(lat_lng_coords[1])

In [96]:
# add coordinates to the DF
toronto_data_clean_grouped['Latitude'] = lats
toronto_data_clean_grouped['Longitude'] = lons

toronto_data_clean_grouped.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


<a id='item3'></a>

## 4. Explore Neighborhoods in Toronto City

In [97]:
# work only with boroughs that contain the word Toronto
toronto_city_data = toronto_data_clean_grouped[toronto_data_clean_grouped['Borough'].str.contains('Toronto')]
toronto_city_data.shape

(38, 5)

#### Define Foursquare Credentials and Version

In [98]:
CLIENT_ID = 'GFRY1XKD0KM34O1PNLXM2PCVPRDDBEKMKH1XBNKSZJDYWUCH' # your Foursquare ID
CLIENT_SECRET = 'LIFCGOJSSBNFZOY2LXGUNIZEVS2QC5TL2OIVYTQZD0IAF12V' # your Foursquare Secret
VERSION = '20190929'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GFRY1XKD0KM34O1PNLXM2PCVPRDDBEKMKH1XBNKSZJDYWUCH
CLIENT_SECRET:LIFCGOJSSBNFZOY2LXGUNIZEVS2QC5TL2OIVYTQZD0IAF12V


#### Let's create a function to get the top 100 venues that are in all the neighborhoods in Toronto City within a radius of 500 meters

In [99]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get nearby venues

In [100]:
toronto_city_venues = getNearbyVenues(names=toronto_city_data['Neighbourhood'],
                                      latitudes=toronto_city_data['Latitude'],
                                      longitudes=toronto_city_data['Longitude'])
toronto_city_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.295425,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676531,-79.295425,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676531,-79.295425,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
3,The Beaches,43.676531,-79.295425,Mastermind Toys,43.671453,-79.293971,Toy / Game Store
4,The Beaches,43.676531,-79.295425,The Beech Tree,43.680493,-79.288846,Gastropub


Let's check how many venues were returned for each neighborhood

In [101]:
toronto_city_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",64,64,64,64,64,64
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,85,85,85,85,85,85


<a id='item5'></a>

## 5. Analyze Each Neighborhood

In [102]:
# one hot encoding
toronto_city_onehot = pd.get_dummies(toronto_city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_city_onehot['Neighbourhood'] = toronto_city_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_city_onehot.columns[-1]] + list(toronto_city_onehot.columns[:-1])
toronto_city_onehot = toronto_city_onehot[fixed_columns]

toronto_city_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [103]:
toronto_city_grouped = toronto_city_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_city_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.0,0.0,0.03,0.0,0.01,0.0,0.00,0.03,...,0.0,0.01,0.0,0.0,0.0,0.01,0.00,0.0,0.0,0.00
1,Berczy Park,0.00,0.0,0.0,0.00,0.0,0.02,0.0,0.00,0.00,...,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.01,0.0,0.0,0.00,0.0,0.03,0.0,0.02,0.00,...,0.0,0.03,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.0,0.0,0.03,0.0,0.01,0.0,0.01,0.02,...,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.01
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.0,0.0,0.01,0.0,0.00,0.0,0.00,0.02,...,0.0,0.00,0.0,0.0,0.0,0.00,0.01,0.0,0.0,0.01


#### Let's print each neighborhood along with the top 5 most common venues

In [104]:
num_top_venues = 5

for hood in toronto_city_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_city_grouped[toronto_city_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.07
2        Hotel  0.06
3   Restaurant  0.04
4   Steakhouse  0.03


----Berczy Park----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.06
2   Restaurant  0.06
3        Hotel  0.05
4     Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.10
1                    Café  0.08
2                     Bar  0.07
3              Restaurant  0.06
4  Furniture / Home Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.06
2        Hotel  0.04
3          Bar  0.04
4   Steakhouse  0.03


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.05
2  Italian Restaurant  0.05
3          Restaurant  0.05
4  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [106]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_city_grouped['Neighbourhood']

for ind in np.arange(toronto_city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Restaurant,Sushi Restaurant,Asian Restaurant,Bar,Steakhouse,American Restaurant,Gastropub
1,Berczy Park,Coffee Shop,Restaurant,Café,Hotel,Beer Bar,Italian Restaurant,Japanese Restaurant,BBQ Joint,Farmers Market,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Bar,Restaurant,Bakery,Furniture / Home Store,Art Gallery,Vegetarian / Vegan Restaurant,Gift Shop,Lounge
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Hotel,Bar,Italian Restaurant,Pizza Place,Steakhouse,Sushi Restaurant,Restaurant,American Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Restaurant,Café,Park,Bakery,Gym,Hotel,Dessert Shop,Spa


<a id='item6'></a>

## 6. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 2 clusters.

In [120]:
# set number of clusters
kclusters = 2

toronto_city_grouped_clustering = toronto_city_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [119]:
# add clustering labels
if not 'Cluster Labels' in neighborhoods_venues_sorted:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
    toronto_city_merged = toronto_city_data.copy()

    # merge neighborhoods_venues_sorted with toronto_city_data to add latitude/longitude for each neighborhood
    toronto_city_merged = toronto_city_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_city_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676531,-79.295425,1,Pub,Bar,Pizza Place,Japanese Restaurant,Bank,Ice Cream Shop,French Restaurant,Bookstore,Juice Bar,Breakfast Spot
41,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105,1,Greek Restaurant,Café,Spa,Ice Cream Shop,Italian Restaurant,Bakery,Yoga Studio,Cosmetics Shop,Business Service,Chinese Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667,1,Coffee Shop,Fast Food Restaurant,Gym,Sandwich Place,Café,Bakery,Brewery,Burrito Place,Breakfast Spot,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.660629,-79.334855,1,Coffee Shop,Bakery,Italian Restaurant,Diner,Bar,Café,American Restaurant,Pizza Place,Pub,Bank
44,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133,1,Bookstore,Coffee Shop,Bus Line,Café,Gym / Fitness Center,Park,College Quad,Trail,Restaurant,Dumpling Restaurant


Let's get the geographical coordinates of Toronto.

In [109]:
address = 'Toronto'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Finally, let's visualize the resulting clusters

In [110]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_city_merged['Latitude'], toronto_city_merged['Longitude'], toronto_city_merged['Neighbourhood'], toronto_city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item7'></a>

## 7. Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

#### Cluster 1

In [116]:
toronto_city_merged.loc[toronto_city_merged['Cluster Labels'] == 0, toronto_city_merged.columns[[1] + list(range(5, toronto_city_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,0,Park,Playground,Gym,Grocery Store,Thai Restaurant,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Farmers Market
50,Downtown Toronto,0,Park,Playground,Candy Store,Trail,Grocery Store,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Dog Run
63,Central Toronto,0,Playground,Pharmacy,Garden,Bank,Cupcake Shop,Doner Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 2

In [117]:
toronto_city_merged.loc[toronto_city_merged['Cluster Labels'] == 1, toronto_city_merged.columns[[1] + list(range(5, toronto_city_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,1,Pub,Bar,Pizza Place,Japanese Restaurant,Bank,Ice Cream Shop,French Restaurant,Bookstore,Juice Bar,Breakfast Spot
41,East Toronto,1,Greek Restaurant,Café,Spa,Ice Cream Shop,Italian Restaurant,Bakery,Yoga Studio,Cosmetics Shop,Business Service,Chinese Restaurant
42,East Toronto,1,Coffee Shop,Fast Food Restaurant,Gym,Sandwich Place,Café,Bakery,Brewery,Burrito Place,Breakfast Spot,Sushi Restaurant
43,East Toronto,1,Coffee Shop,Bakery,Italian Restaurant,Diner,Bar,Café,American Restaurant,Pizza Place,Pub,Bank
44,Central Toronto,1,Bookstore,Coffee Shop,Bus Line,Café,Gym / Fitness Center,Park,College Quad,Trail,Restaurant,Dumpling Restaurant
45,Central Toronto,1,Dog Run,Park,Pizza Place,Bar,Breakfast Spot,Dessert Shop,Brewery,Sushi Restaurant,Pharmacy,Sandwich Place
46,Central Toronto,1,Sporting Goods Shop,Café,Coffee Shop,Restaurant,Diner,Clothing Store,Italian Restaurant,Salon / Barbershop,Flower Shop,Spa
47,Central Toronto,1,Pizza Place,Dessert Shop,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Gym,Seafood Restaurant,Gastropub
49,Central Toronto,1,Coffee Shop,Italian Restaurant,Pub,Sushi Restaurant,Pharmacy,Skating Rink,Restaurant,Bagel Shop,Pizza Place,Gym
51,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Pizza Place,Grocery Store,Indian Restaurant,Thai Restaurant,Japanese Restaurant,Pub,Bakery


<a id='item7'></a>

## 8. Conclusion

- **Cluster 1:** There are 3 Neighbourhoods where the main spots are green urban areas (parks, gardens, and so on). Due to this, we can name this cluster as **Green Spaces**
- **Cluster 2:** The rest of the Neighbourhoods are mainly dominated by places where people can have some coffee. Therefore, we can name this cluster as **Coffee Areas**